# Polars Experiments



In [ ]:
import  numpy as np
import polars as pl
from polars import col

from mintalib.core import calc_sma, calc_atr, calc_macd
from mintalib.indicators import SMA, ATR, MACD


In [135]:
from mintalib.samples import sample_prices

prices = sample_prices()
prices = pl.from_pandas(prices, include_index=True)
prices

# Links
# https://docs.pola.rs/user-guide/expressions/user-defined-python-functions/#combining-multiple-column-values

date,open,high,low,close,volume
datetime[ns],f64,f64,f64,f64,i64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600
…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800


In [136]:
pstruct = prices.to_struct("prices")
pstruct


prices
struct[6]
"{1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600}"
"{1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800}"
"{1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000}"
"{1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600}"
"{1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600}"
…
"{2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400}"
"{2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200}"
"{2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800}"


In [137]:
from polars.datatypes import Struct, Float64


macd_type = Struct({'macd': Float64, 'macdsignal': Float64, 'macdhist': Float64})

def my_sma(series, period: int = 20):
    print(type(series).__name__)
    return calc_sma(series, period)

def my_atr(prices, period: int = 14):
    print(type(prices).__name__)
    if hasattr(prices, 'dtype') and prices.dtype.__class__.__name__ == 'Struct':
        print("struct ...")
        prices = prices.struct
    return calc_atr(prices, period)

def my_macd(series):
    print(type(series).__name__)
    result = calc_macd(series, wrap=True)
    if hasattr(result, 'to_struct'):
        return result.to_struct("macd")



df = prices.with_columns(
    col('close').map_batches(my_sma).alias("sma"),
    col('close').map_batches(my_macd).alias("macd"),
    pl.struct(pl.all()).map_batches(my_atr).alias("atr")
#    col('close').map_batches(my_macd)
#    sma = col('close').map_batches(my_sma)
#     atr = pl.struct(pl.all()).map_batches(my_atr)
#    macd = col('close').map_batches(my_macd)
#    atr = pl.all().map_batches(my_atr)
#    MACD() @ col("close").map_batches(calc_macd),
#    atr = ATR() @ pl.all()
#    sma = col('close').map_batches(SMA(20))
#    atr = ATR() @ pl.all()
#    sma = SMA(20) @ pl.all()
#    sma = SMA(20) @ col('close')
)

df

SeriesSeries
Series
struct ...



date,open,high,low,close,volume,sma,macd,atr
datetime[ns],f64,f64,f64,f64,i64,f64,struct[3],f64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600,NaN,"{NaN,NaN,NaN}",NaN
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800,NaN,"{NaN,NaN,NaN}",NaN
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000,NaN,"{NaN,NaN,NaN}",NaN
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600,NaN,"{NaN,NaN,NaN}",NaN
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600,NaN,"{NaN,NaN,NaN}",NaN
…,…,…,…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400,227.524,"{1.815958,1.313965,0.501993}",4.516121
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200,228.0785,"{1.941114,1.439395,0.501719}",4.479971
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800,228.2425,"{2.046565,1.560829,0.485736}",4.39783


In [138]:
df.dtypes


[Datetime(time_unit='ns', time_zone=None),
 Float64,
 Float64,
 Float64,
 Float64,
 Int64,
 Float64,
 Struct({'macd': Float64, 'macdsignal': Float64, 'macdhist': Float64}),
 Float64]

In [139]:
sma = calc_sma(prices['close'], 20, wrap=True)
sma

""
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [140]:
pl.Series(values=sma, name=None)

""
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [141]:
MACD() @ prices

macd,macdsignal,macdhist
f64,f64,f64
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [142]:
ATR() @ prices

""
f64
NaN
NaN
NaN
NaN
NaN
…
4.516121
4.479971
4.39783


In [143]:
SMA(20) @ prices['close']

""
f64
NaN
NaN
NaN
NaN
NaN
…
227.524
228.0785
228.2425


In [144]:
prices.select(
    pl.col('open', 'close').map_batches(my_sma)
)

SeriesSeries



open,close
f64,f64
NaN,NaN
NaN,NaN
NaN,NaN
NaN,NaN
NaN,NaN
…,…
227.055,227.524
227.7575,228.0785
228.179499,228.2425


In [145]:
prices.select(
    pl.col(['open', 'close']).map_batches(my_sma)
)

SeriesSeries



open,close
f64,f64
NaN,NaN
NaN,NaN
NaN,NaN
NaN,NaN
NaN,NaN
…,…
227.055,227.524
227.7575,228.0785
228.179499,228.2425


In [146]:
prices.select(
    pl.col('close').map_batches(my_macd, macd_type).struct.unnest()
)

SeriesSeries

Series


macd,macdsignal,macdhist
f64,f64,f64
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
NaN,NaN,NaN
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [147]:
df.select(
    pl.col('macd').struct[0]
)

macd
f64
NaN
NaN
NaN
NaN
NaN
…
1.815958
1.941114
2.046565


In [151]:
prices.select(pl.struct("*").alias("dohlcv"))

dohlcv
struct[6]
"{1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600}"
"{1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800}"
"{1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000}"
"{1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600}"
"{1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600}"
…
"{2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400}"
"{2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200}"
"{2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800}"
